## Chess Analysis
**The objective of this analysis is to perform data parsing on a collection of my chess games, followed by data cleansing, and ultimately, to present the cleaned data in a visual format using Tableau.**

In [1]:
import re
import pandas as pd
import os
import requests
import json
import urllib.request
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

from pathlib import Path
from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
pgnMeta = ["Event","Site","Date","Round","White","Black","Result", \
            "CurrentPosition","Timezone", "ECO","ECOURL","UTDate","UTCTime","WhiteELO", \
            "BlackELO","Timecontrol","Termination","StartTime","EndDate","EndTime","Link","Moves"]
tgtFilePath="./Documents/mygames.csv" #This is the path where the final CSV gets created
moveStartLine = 22 #Moves in chess.com PGNs typically start from the 22nd line for each game
PGNDirectory="./Documents/PGN" #This is the location where the API downloads the PGNs from the archives
user='vilja228' #The user for whom the script is intended to run


def getPGN(user):
    """This function accesses the chess.com public API and downloads all the PGNs to a folder"""
    pgn_archive_links = requests.get("https://api.chess.com/pub/player/vilja228/games/archives", verify=False)
    if not os.path.exists(PGNDirectory):
        os.makedirs(PGNDirectory)

    for url in json.loads(pgn_archive_links.content)["archives"]:
        filepath = PGNDirectory + "/"+ url.split("/")[7]+url.split("/")[8]+'.pgn'
        my_file = Path(filepath)
        if not my_file.is_file():
            urllib.request.urlretrieve(url+'/pgn',filepath)

def importPGNData(filepath):
    """This function returns the data read as a string"""
    with open(filepath) as f:
        return f.readlines()

def getEdgePoints(data):
    """This function returns the start and end indices for each game in the PGN"""
    ends=[]
    starts=[]
    for n,l in enumerate(data):
        if l.startswith("[Event"):
            if n!=0:
                ends.append(n - 1)
            starts.append(n)
        elif (n==len(data)-1):
            ends.append(n)

    return (starts,ends)


def grpGames(data, starts, ends):
    """This function groups games into individual lists based on the start and end index"""
    blocks=[]
    for i in range(len(ends)):
        try:
            element = data[starts[i]: ends[i]+1]
        except:
            print(i)
        if element not in blocks: blocks.append(element)
    return blocks

def mergeMoves(game):
    """This function cleans out the moves and other attributes, removes newlines and formats the list to be converted into a dictionary"""
    firstmove=lastmove=-1
    for n,eachrow in enumerate(game):
                game[n]=game[n].replace('\n','')
                try:
                    if n <= moveStartLine-2: game[n] = stripwhitespace(game[n]).split('~')[1].strip(']["')
                except:
                    if n <= moveStartLine-4: game[n] = stripwhitespace(game[n]).split('~')[1].strip(']["')
                    pass
    return list(filter(None,game))


def stripwhitespace(text):
    lst = text.split('"')
    for i, item in enumerate(lst):
        if not i % 2:
            lst[i] = re.sub("\s+", "~", item)
    return '"'.join(lst)

def createGameDictLetsPlay(game_dict):
    """This is a helper function to address games under Lets Play events on chess.com. These events have a slightly different way of representation than the Live Chess events"""
    for n, move in enumerate(game_dict["Moves"].split(" ")):

        if n%3==0: #every 3rd element is the move number
            if move == '1-0' or move=='0-1' or move=='1/2-1/2':
                None
            else: movenum = n
        elif n==movenum+2:
            if move == '1-0' or move=='0-1' or move=='1/2-1/2':
                None
            else: game_dict["whitemoves"].append(move)
        else:
            if move == '1-0' or move=='0-1' or move=='1/2-1/2':
                None
            else: game_dict["blackmoves"].append(move)

    if len(game_dict["blackmoves"])>len(game_dict["whitemoves"]): game_dict["whitemoves"].append("over")
    if len(game_dict["blackmoves"])<len(game_dict["whitemoves"]): game_dict["blackmoves"].append("over")
    del game_dict["Moves"]
    return game_dict


def createGameDictLiveChess(game_dict):
    """This is a helper function to address games under Live Chess events on chess.com."""
    try:
        for n, move in enumerate(game_dict["Moves"].split(" ")):

            if '{' in move or '}' in move:
                None
            elif '.' in move:
                movenum = int(move.split(".")[0])
                if "..." in move:
                    color = 'black'
                else: color="white"
            else:
                if color=="white":
                    if move == '1-0' or move=='0-1' or move=='1/2-1/2': None
                    else: game_dict["whitemoves"].append(move)
                else:
                    if move == '1-0' or move=='0-1' or move=='1/2-1/2': None
                    else: game_dict["blackmoves"].append(move)

        if len(game_dict["blackmoves"])>len(game_dict["whitemoves"]): game_dict["whitemoves"].append("over")
        if len(game_dict["blackmoves"])<len(game_dict["whitemoves"]): game_dict["blackmoves"].append("over")
        del game_dict["Moves"]
    except: pass

    return game_dict


def createGameDict(games):
    allgames=[]
    for gamenum, eachgame in enumerate(games):
        game_dict = dict(zip(pgnMeta, eachgame))
        movenum = 0
        game_dict["whitemoves"] = []
        game_dict["blackmoves"] = []
        color="white"
        if game_dict["Event"]=="Let's Play!": allgames.append(createGameDictLetsPlay(game_dict))
        else: allgames.append(createGameDictLiveChess(game_dict))

    return allgames



def main():
    getPGN(user)
    tgtFilePathObj = Path(tgtFilePath)
    tgtFilePathObj.unlink(missing_ok=True)

    with os.scandir(PGNDirectory) as pgndir:
        for file in pgndir:
            print('*', end =" ")
            data = importPGNData(file)

            starts, ends = getEdgePoints(data)
            games = grpGames(data, starts, ends)
            games = list(map(mergeMoves, games))
            allgames= createGameDict(games)

            for gamenum, game in enumerate(allgames):
                df = pd.DataFrame(allgames[gamenum])

                with open(tgtFilePath, 'a') as f:
                    df.to_csv(f, mode='a', header=f.tell()==0)
    print("Export Complete!")

#Run Program
main()


* * * * * * * * * * * * * * * * * * * * * * * * * * * * Export Complete!


In [2]:
df = pd.read_csv("./Documents/mygames.csv")
df.head(2)

/var/folders/h0/2jwjxccn5sn94dwgppl4kfhc0000gn/T/ipykernel_13166/1793610034.py:1: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./Documents/mygames.csv")


,Unnamed: 0,Event,Site,Date,Round,White,Black,Result,CurrentPosition,Timezone,...,WhiteELO,BlackELO,Timecontrol,Termination,StartTime,EndDate,EndTime,Link,whitemoves,blackmoves
0,0,Live Chess,Chess.com,2021.06.30,-,vilja228,Andersonkazazi,0-1,r4rk1/p1pp1pnp/1p4pq/3N1B2/4P3/3P1P2/PP3P2/5KR...,UTC,...,1276,1348,300,Andersonkazazi won by resignation,21:35:36,2021.06.30,21:41:21,https://www.chess.com/game/live/18828012117,d4,e5
1,1,Live Chess,Chess.com,2021.06.30,-,vilja228,Andersonkazazi,0-1,r4rk1/p1pp1pnp/1p4pq/3N1B2/4P3/3P1P2/PP3P2/5KR...,UTC,...,1276,1348,300,Andersonkazazi won by resignation,21:35:36,2021.06.30,21:41:21,https://www.chess.com/game/live/18828012117,dxe5,Nc6


## Cleaning Data

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232332 entries, 0 to 232331
Data columns (total 24 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Unnamed: 0       232332 non-null  int64 
 1   Event            232332 non-null  object
 2   Site             232332 non-null  object
 3   Date             232332 non-null  object
 4   Round            232332 non-null  object
 5   White            232332 non-null  object
 6   Black            232332 non-null  object
 7   Result           232332 non-null  object
 8   CurrentPosition  232332 non-null  object
 9   Timezone         232332 non-null  object
 10  ECO              232332 non-null  object
 11  ECOURL           232332 non-null  object
 12  UTDate           232332 non-null  object
 13  UTCTime          232332 non-null  object
 14  WhiteELO         232332 non-null  int64 
 15  BlackELO         232332 non-null  int64 
 16  Timecontrol      232332 non-null  object
 17  Terminatio

In [4]:
df.rename( columns={'Unnamed: 0':'Moves'}, inplace=True )

In [5]:
df.index

RangeIndex(start=0, stop=232332, step=1)

In [6]:
df.columns

Index(['Moves', 'Event', 'Site', 'Date', 'Round', 'White', 'Black', 'Result',
       'CurrentPosition', 'Timezone', 'ECO', 'ECOURL', 'UTDate', 'UTCTime',
       'WhiteELO', 'BlackELO', 'Timecontrol', 'Termination', 'StartTime',
       'EndDate', 'EndTime', 'Link', 'whitemoves', 'blackmoves'],
      dtype='object')

In [8]:
df.dtypes

Moves               int64
Event              object
Site               object
Date               object
Round              object
White              object
Black              object
Result             object
CurrentPosition    object
Timezone           object
ECO                object
ECOURL             object
UTDate             object
UTCTime            object
WhiteELO            int64
BlackELO            int64
Timecontrol        object
Termination        object
StartTime          object
EndDate            object
EndTime            object
Link               object
whitemoves         object
blackmoves         object
dtype: object

In [9]:
df['Timecontrol'] = df['Timecontrol'].replace(['180+2'],'182')
df['Timecontrol'] = df['Timecontrol'].replace(['120+1'],'121')
df['Timecontrol'] = df['Timecontrol'].replace(['1/259200'],'259200')
df['Timecontrol'] = df['Timecontrol'].replace(['1/86400'],'86400')
df['Timecontrol'] = df['Timecontrol'].replace(['60+1'],'61')
df

,Moves,Event,Site,Date,Round,White,Black,Result,CurrentPosition,Timezone,...,WhiteELO,BlackELO,Timecontrol,Termination,StartTime,EndDate,EndTime,Link,whitemoves,blackmoves
0,0,Live Chess,Chess.com,2021.06.30,-,vilja228,Andersonkazazi,0-1,r4rk1/p1pp1pnp/1p4pq/3N1B2/4P3/3P1P2/PP3P2/5KR...,UTC,...,1276,1348,300,Andersonkazazi won by resignation,21:35:36,2021.06.30,21:41:21,https://www.chess.com/game/live/18828012117,d4,e5
1,1,Live Chess,Chess.com,2021.06.30,-,vilja228,Andersonkazazi,0-1,r4rk1/p1pp1pnp/1p4pq/3N1B2/4P3/3P1P2/PP3P2/5KR...,UTC,...,1276,1348,300,Andersonkazazi won by resignation,21:35:36,2021.06.30,21:41:21,https://www.chess.com/game/live/18828012117,dxe5,Nc6
2,2,Live Chess,Chess.com,2021.06.30,-,vilja228,Andersonkazazi,0-1,r4rk1/p1pp1pnp/1p4pq/3N1B2/4P3/3P1P2/PP3P2/5KR...,UTC,...,1276,1348,300,Andersonkazazi won by resignation,21:35:36,2021.06.30,21:41:21,https://www.chess.com/game/live/18828012117,Nf3,Qe7
3,3,Live Chess,Chess.com,2021.06.30,-,vilja228,Andersonkazazi,0-1,r4rk1/p1pp1pnp/1p4pq/3N1B2/4P3/3P1P2/PP3P2/5KR...,UTC,...,1276,1348,300,Andersonkazazi won by resignation,21:35:36,2021.06.30,21:41:21,https://www.chess.com/game/live/18828012117,Nc3,Nxe5
4,4,Live Chess,Chess.com,2021.06.30,-,vilja228,Andersonkazazi,0-1,r4rk1/p1pp1pnp/1p4pq/3N1B2/4P3/3P1P2/PP3P2/5KR...,UTC,...,1276,1348,300,Andersonkazazi won by resignation,21:35:36,2021.06.30,21:41:21,https://www.chess.com/game/live/18828012117,Bf4,Nxf3+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232327,37,Live Chess,Chess.com,2021.08.01,-,ReyRubio115,vilja228,0-1,5k2/qn4p1/2Q2p1p/4pP2/2Bp2P1/p4P1P/2P3K1/8 w - -,UTC,...,1388,1357,300,vilja228 won on time,22:53:03,2021.08.01,23:03:22,https://www.chess.com/game/live/21597561475,Qb4,h6
232328,38,Live Chess,Chess.com,2021.08.01,-,ReyRubio115,vilja228,0-1,5k2/qn4p1/2Q2p1p/4pP2/2Bp2P1/p4P1P/2P3K1/8 w - -,UTC,...,1388,1357,300,vilja228 won on time,22:53:03,2021.08.01,23:03:22,https://www.chess.com/game/live/21597561475,Bc4,Qa7
232329,39,Live Chess,Chess.com,2021.08.01,-,ReyRubio115,vilja228,0-1,5k2/qn4p1/2Q2p1p/4pP2/2Bp2P1/p4P1P/2P3K1/8 w - -,UTC,...,1388,1357,300,vilja228 won on time,22:53:03,2021.08.01,23:03:22,https://www.chess.com/game/live/21597561475,Kg2,Kf8
232330,40,Live Chess,Chess.com,2021.08.01,-,ReyRubio115,vilja228,0-1,5k2/qn4p1/2Q2p1p/4pP2/2Bp2P1/p4P1P/2P3K1/8 w - -,UTC,...,1388,1357,300,vilja228 won on time,22:53:03,2021.08.01,23:03:22,https://www.chess.com/game/live/21597561475,Qb5,a3


In [10]:
df['Date' ] = pd.to_datetime(df['Date'])
df['UTDate' ] = pd.to_datetime(df['UTDate'])
df['EndDate' ] = pd.to_datetime(df['EndDate'])
df = df.astype({'Timecontrol':'int'})

In [11]:
df = df.astype({'Timecontrol':'int'})

In [12]:
df['StartTime'] = pd.to_datetime(df['StartTime'], format= '%H:%M:%S')
df['EndTime'] = pd.to_datetime(df['EndTime'], format= '%H:%M:%S')

### <a href="https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior">Set Times</a>

In [13]:
df.drop(columns=['Event', 'Round', 'Timezone', 'Link', 'Site', 'ECOURL'], inplace=True) ## Dropping unessasary coulumns

In [14]:
df

,Moves,Date,White,Black,Result,CurrentPosition,ECO,UTDate,UTCTime,WhiteELO,BlackELO,Timecontrol,Termination,StartTime,EndDate,EndTime,whitemoves,blackmoves
0,0,2021-06-30,vilja228,Andersonkazazi,0-1,r4rk1/p1pp1pnp/1p4pq/3N1B2/4P3/3P1P2/PP3P2/5KR...,A40,2021-06-30,21:35:36,1276,1348,300,Andersonkazazi won by resignation,1900-01-01 21:35:36,2021-06-30,1900-01-01 21:41:21,d4,e5
1,1,2021-06-30,vilja228,Andersonkazazi,0-1,r4rk1/p1pp1pnp/1p4pq/3N1B2/4P3/3P1P2/PP3P2/5KR...,A40,2021-06-30,21:35:36,1276,1348,300,Andersonkazazi won by resignation,1900-01-01 21:35:36,2021-06-30,1900-01-01 21:41:21,dxe5,Nc6
2,2,2021-06-30,vilja228,Andersonkazazi,0-1,r4rk1/p1pp1pnp/1p4pq/3N1B2/4P3/3P1P2/PP3P2/5KR...,A40,2021-06-30,21:35:36,1276,1348,300,Andersonkazazi won by resignation,1900-01-01 21:35:36,2021-06-30,1900-01-01 21:41:21,Nf3,Qe7
3,3,2021-06-30,vilja228,Andersonkazazi,0-1,r4rk1/p1pp1pnp/1p4pq/3N1B2/4P3/3P1P2/PP3P2/5KR...,A40,2021-06-30,21:35:36,1276,1348,300,Andersonkazazi won by resignation,1900-01-01 21:35:36,2021-06-30,1900-01-01 21:41:21,Nc3,Nxe5
4,4,2021-06-30,vilja228,Andersonkazazi,0-1,r4rk1/p1pp1pnp/1p4pq/3N1B2/4P3/3P1P2/PP3P2/5KR...,A40,2021-06-30,21:35:36,1276,1348,300,Andersonkazazi won by resignation,1900-01-01 21:35:36,2021-06-30,1900-01-01 21:41:21,Bf4,Nxf3+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232327,37,2021-08-01,ReyRubio115,vilja228,0-1,5k2/qn4p1/2Q2p1p/4pP2/2Bp2P1/p4P1P/2P3K1/8 w - -,A00,2021-08-01,22:53:03,1388,1357,300,vilja228 won on time,1900-01-01 22:53:03,2021-08-01,1900-01-01 23:03:22,Qb4,h6
232328,38,2021-08-01,ReyRubio115,vilja228,0-1,5k2/qn4p1/2Q2p1p/4pP2/2Bp2P1/p4P1P/2P3K1/8 w - -,A00,2021-08-01,22:53:03,1388,1357,300,vilja228 won on time,1900-01-01 22:53:03,2021-08-01,1900-01-01 23:03:22,Bc4,Qa7
232329,39,2021-08-01,ReyRubio115,vilja228,0-1,5k2/qn4p1/2Q2p1p/4pP2/2Bp2P1/p4P1P/2P3K1/8 w - -,A00,2021-08-01,22:53:03,1388,1357,300,vilja228 won on time,1900-01-01 22:53:03,2021-08-01,1900-01-01 23:03:22,Kg2,Kf8
232330,40,2021-08-01,ReyRubio115,vilja228,0-1,5k2/qn4p1/2Q2p1p/4pP2/2Bp2P1/p4P1P/2P3K1/8 w - -,A00,2021-08-01,22:53:03,1388,1357,300,vilja228 won on time,1900-01-01 22:53:03,2021-08-01,1900-01-01 23:03:22,Qb5,a3


In [15]:
df.dtypes

Moves                       int64
Date               datetime64[ns]
White                      object
Black                      object
Result                     object
CurrentPosition            object
ECO                        object
UTDate             datetime64[ns]
UTCTime                    object
WhiteELO                    int64
BlackELO                    int64
Timecontrol                 int64
Termination                object
StartTime          datetime64[ns]
EndDate            datetime64[ns]
EndTime            datetime64[ns]
whitemoves                 object
blackmoves                 object
dtype: object

In [16]:
pd.to_datetime(df['Date']).head()

0   2021-06-30
1   2021-06-30
2   2021-06-30
3   2021-06-30
4   2021-06-30
Name: Date, dtype: datetime64[ns]

In [17]:
pd.isnull(df).sum()

Moves              0
Date               0
White              0
Black              0
Result             0
CurrentPosition    0
ECO                0
UTDate             0
UTCTime            0
WhiteELO           0
BlackELO           0
Timecontrol        0
Termination        0
StartTime          0
EndDate            0
EndTime            0
whitemoves         0
blackmoves         0
dtype: int64

In [18]:
df['Result'].unique()

array(['0-1', '1-0', '1/2-1/2'], dtype=object)

In [19]:
df['Result'].value_counts()

1-0        111497
0-1        105013
1/2-1/2     15822
Name: Result, dtype: int64

In [20]:
df['Termination'].unique

<bound method Series.unique of 0         Andersonkazazi won by resignation
1         Andersonkazazi won by resignation
2         Andersonkazazi won by resignation
3         Andersonkazazi won by resignation
4         Andersonkazazi won by resignation
                        ...                
232327                 vilja228 won on time
232328                 vilja228 won on time
232329                 vilja228 won on time
232330                 vilja228 won on time
232331                 vilja228 won on time
Name: Termination, Length: 232332, dtype: object>

In [21]:
conditions = \
    [(df["White"] == user) & (df["Result"] == "1-0"), 
     (df["White"] == user) & (df["Result"] == "0-1"), 
     (df["White"] != user) & (df["Result"] == "1-0"), 
     (df["White"] != user) & (df["Result"] == "0-1"), 
     (df["White"] == user) & (df["Result"] == "1/2-1/2"),
     (df["White"] != user) & (df["Result"] == "1/2-1/2")]
    
values = ["Win", "Loss", "Loss", "Win", "Draw", "Draw"]
                
df['my_result'] = np.select(conditions, values)
df.head()

,Moves,Date,White,Black,Result,CurrentPosition,ECO,UTDate,UTCTime,WhiteELO,BlackELO,Timecontrol,Termination,StartTime,EndDate,EndTime,whitemoves,blackmoves,my_result
0,0,2021-06-30,vilja228,Andersonkazazi,0-1,r4rk1/p1pp1pnp/1p4pq/3N1B2/4P3/3P1P2/PP3P2/5KR...,A40,2021-06-30,21:35:36,1276,1348,300,Andersonkazazi won by resignation,1900-01-01 21:35:36,2021-06-30,1900-01-01 21:41:21,d4,e5,Loss
1,1,2021-06-30,vilja228,Andersonkazazi,0-1,r4rk1/p1pp1pnp/1p4pq/3N1B2/4P3/3P1P2/PP3P2/5KR...,A40,2021-06-30,21:35:36,1276,1348,300,Andersonkazazi won by resignation,1900-01-01 21:35:36,2021-06-30,1900-01-01 21:41:21,dxe5,Nc6,Loss
2,2,2021-06-30,vilja228,Andersonkazazi,0-1,r4rk1/p1pp1pnp/1p4pq/3N1B2/4P3/3P1P2/PP3P2/5KR...,A40,2021-06-30,21:35:36,1276,1348,300,Andersonkazazi won by resignation,1900-01-01 21:35:36,2021-06-30,1900-01-01 21:41:21,Nf3,Qe7,Loss
3,3,2021-06-30,vilja228,Andersonkazazi,0-1,r4rk1/p1pp1pnp/1p4pq/3N1B2/4P3/3P1P2/PP3P2/5KR...,A40,2021-06-30,21:35:36,1276,1348,300,Andersonkazazi won by resignation,1900-01-01 21:35:36,2021-06-30,1900-01-01 21:41:21,Nc3,Nxe5,Loss
4,4,2021-06-30,vilja228,Andersonkazazi,0-1,r4rk1/p1pp1pnp/1p4pq/3N1B2/4P3/3P1P2/PP3P2/5KR...,A40,2021-06-30,21:35:36,1276,1348,300,Andersonkazazi won by resignation,1900-01-01 21:35:36,2021-06-30,1900-01-01 21:41:21,Bf4,Nxf3+,Loss


In [22]:
df = df.astype({'White':'string','Black':'string'})
df = df.astype({'whitemoves':'string','blackmoves':'string'})

In [23]:
filt = ((df["Timecontrol"] >= 180) | (df["Timecontrol"] <= 300))
results = df.loc[filt]
results.loc[(((df['whitemoves'].str.contains('[a-z]8=Q', regex=True)) & (df['White'] == user) | (df['blackmoves'].str.contains('[a-z]8=Q', regex=True)) & (df['Black'] == user)))]

,Moves,Date,White,Black,Result,CurrentPosition,ECO,UTDate,UTCTime,WhiteELO,BlackELO,Timecontrol,Termination,StartTime,EndDate,EndTime,whitemoves,blackmoves,my_result
1227,57,2021-06-14,vilja228,SoloCrew,1-0,8/8/8/4NK2/5pPk/5P2/7Q/8 b - -,D02,2021-06-14,16:21:50,1285,1306,300,vilja228 won by checkmate,1900-01-01 16:21:50,2021-06-14,1900-01-01 16:30:56,d8=Q,Kf2,Win
1327,48,2021-06-14,vilja228,aykutgokce91,0-1,7q/5K2/8/8/8/6P1/2k5/8 w - -,A40,2021-06-14,08:07:11,1279,1336,300,aykutgokce91 won by resignation,1900-01-01 08:07:11,2021-06-14,1900-01-01 08:14:02,h8=Q,Qc3+,Loss
2897,39,2021-06-08,vilja228,SMNAGAR,1-0,5kb1/4Q3/2p1pK2/1pPpPp2/pP6/P3P3/8/8 b - -,A40,2021-06-08,09:01:43,1337,1335,300,vilja228 won by checkmate,1900-01-01 09:01:43,2021-06-08,1900-01-01 09:09:02,g8=Q,Bf7,Win
3159,33,2021-06-07,vilja228,tumbaburros,1-0,8/p7/4pQ1p/4N2k/P5P1/3B4/5P2/6K1 b - -,A40,2021-06-07,19:10:24,1338,1318,300,vilja228 won by checkmate,1900-01-01 19:10:24,2021-06-07,1900-01-01 19:19:53,d8=Q+,Kg7,Win
3532,45,2021-06-06,vilja228,eugeniozanforlini,1-0,2k3Q1/1pb5/p7/8/4pP2/1P2P3/P6K/4R3 b - -,D06,2021-06-06,23:01:44,1384,1378,300,vilja228 won by resignation,1900-01-01 23:01:44,2021-06-06,1900-01-01 23:10:19,g8=Q+,over,Win
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229016,44,2021-08-11,vilja228,ibo0206,0-1,2q3K1/q7/8/8/6k1/6P1/8/8 w - -,A40,2021-08-11,17:59:15,1302,1270,300,ibo0206 won by checkmate,1900-01-01 17:59:15,2021-08-11,1900-01-01 18:07:19,h8=Q,Qd3+,Loss
229640,62,2021-08-10,vilja228,MOCKBA_CCCP,1-0,8/8/8/8/8/2K5/1Q6/1k6 b - -,D20,2021-08-10,22:27:26,1310,1295,300,vilja228 won by checkmate,1900-01-01 22:27:26,2021-08-10,1900-01-01 22:36:50,g8=Q,Kd4,Win
229710,53,2021-08-10,vilja228,serbanleo,1-0,8/8/6Q1/3p3K/1R1P4/P7/2k5/8 b - -,A40,2021-08-10,21:12:47,1302,1266,300,vilja228 won by resignation,1900-01-01 21:12:47,2021-08-10,1900-01-01 21:21:11,e8=Q,Kc2,Win
230044,50,2021-08-10,vilja228,fero-tt,1-0,1Q6/8/6K1/8/8/3k4/8/8 b - -,A40,2021-08-10,19:22:33,1268,1272,300,vilja228 won on time,1900-01-01 19:22:33,2021-08-10,1900-01-01 19:32:58,h8=Q,Kb3,Win


In [24]:
df['my_result'].value_counts()

Win     116609
Loss     99901
Draw     15822
Name: my_result, dtype: int64

In [25]:
new_df = df.loc[(df['Moves'] == 1) & ((df['White'] == user) | (df['Black'] == user)) & ((df["Timecontrol"] >= 180) | (df["Timecontrol"] <= 300))]

new_df.to_csv('filtered.csv')


In [27]:
df[df["White"]== user].groupby(["Moves", "my_result"])["my_result"].count()

Moves  my_result
0      Draw          163
       Loss         1571
       Win          1807
1      Draw          163
       Loss         1571
                    ... 
100    Win             1
101    Draw            1
       Win             1
102    Draw            1
       Win             1
Name: my_result, Length: 299, dtype: int64

In [28]:
filt = (df['Moves'] == 1) & ((df['White'] == user) | (df['Black'] == user)) & ((df["Timecontrol"] >= 180) | (df["Timecontrol"] <= 300))


In [29]:
df[df['Black']=='vilja228']['whitemoves']

18          d4
19         Bf4
20          e3
21         Nf3
22        dxe5
          ... 
232327     Qb4
232328     Bc4
232329     Kg2
232330     Qb5
232331     Qc6
Name: whitemoves, Length: 116617, dtype: string

In [31]:
df = df.loc[df['Moves'] == 0]

In [33]:
df['my_result'].value_counts()

Win     3487
Loss    3291
Draw     320
Name: my_result, dtype: int64